# Fixtures

In [1]:
# Plain pytest does not work inside notebooks.
import ipytest
ipytest.autoconfig()

In [14]:
from typing import Iterable

import pytest

## Test dependent fixtures

In [3]:
@pytest.fixture(name="a")
def a_fixture() -> str:
    return "a"

@pytest.fixture(name="b")
def b_fixture(a: str) -> str:
    return f"{a} -> b"

@pytest.fixture(name="d")
def d_fixture(a: str) -> str:
    return f"{a} -> d"

@pytest.fixture(name="c")
def c_fixture(request: pytest.FixtureRequest) -> str:
    return f"{request.getfixturevalue(request.param)} -> c"

In [10]:
%%ipytest -vv

def test_a(a: str) -> None:
    assert a == "a"

def test_b(b: str) -> None:
    assert b == "a -> b"

def test_d(d: str) -> None:
    assert d == "a -> d"

@pytest.mark.parametrize('c', ['a'], indirect=True)
def test_ac(c: str) -> None:
    assert c == "a -> c"

@pytest.mark.parametrize('c', ['a'], indirect=True)
def test_c_multiple(c: str) -> None:
    assert c == "a -> c"

======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-8.3.3, pluggy-1.5.0 -- /home/vbrozik/dev/python-ntb/.venv/bin/python3
cachedir: .pytest_cache
rootdir: /home/vbrozik/dev/python-ntb/explore_pytest
plugins: anyio-4.6.2.post1
collecting ... collected 4 items

t_efce9a9f272c49c5a6a60c470b7cf830.py::test_a PASSED                                         [ 25%]
t_efce9a9f272c49c5a6a60c470b7cf830.py::test_b PASSED                                         [ 50%]
t_efce9a9f272c49c5a6a60c470b7cf830.py::test_d PASSED                                         [ 75%]
t_efce9a9f272c49c5a6a60c470b7cf830.py::test_ac[a] PASSED                                     [100%]

======================================== 4 passed in 0.02s =========================================


## Parametrizing

In [41]:
def process_items(numbers: Iterable[int]) -> list[int]:
    return [item + 1 for item in numbers]

@pytest.fixture(params=[
    {"input": [1, 2, 3], "expected": [2, 3, 4]},
    {"input": [0, 0, 0], "expected": [1, 1, 1]},
])
def inputs_results1_fixture(request):
    return request.param

@pytest.fixture(params=[
    pytest.param(
        ([1, 2, 3], [2, 3, 4]), id='values123'),
    pytest.param(
        ([0, 0, 0], [1, 1, 1]), id='values000'),
    pytest.param(
        ([3, 4, 5], [1, 1, 1]), id='values345', marks=[pytest.mark.skip(reason='Test skipping')]),
    pytest.param(
        ([0, 0, 0], [0, 0, 0]), id='values000xfail', marks=[pytest.mark.xfail(reason='Test xfail', strict=True)]),
])
def inputs_results2_fixture(request) -> tuple[list[int], list[int]]:
    return request.param

In [42]:
%%ipytest -vv

def test_inputs_results1(inputs_results1_fixture) -> None:
    processed_items = process_items(inputs_results1_fixture["input"])
    expected_items = inputs_results1_fixture["expected"]
    
    assert processed_items == expected_items

def test_inputs_results2(inputs_results2_fixture) -> None:
    processed_items = process_items(inputs_results2_fixture[0])
    expected_items = inputs_results2_fixture[1]
    
    assert processed_items == expected_items

======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-8.3.3, pluggy-1.5.0 -- /home/vbrozik/dev/python-ntb/.venv/bin/python3
cachedir: .pytest_cache
rootdir: /home/vbrozik/dev/python-ntb/explore_pytest
plugins: anyio-4.6.2.post1
collecting ... collected 6 items

t_efce9a9f272c49c5a6a60c470b7cf830.py::test_inputs_results1[inputs_results1_fixture0] PASSED [ 16%]
t_efce9a9f272c49c5a6a60c470b7cf830.py::test_inputs_results1[inputs_results1_fixture1] PASSED [ 33%]
t_efce9a9f272c49c5a6a60c470b7cf830.py::test_inputs_results2[values123] PASSED                [ 50%]
t_efce9a9f272c49c5a6a60c470b7cf830.py::test_inputs_results2[values000] PASSED                [ 66%]
t_efce9a9f272c49c5a6a60c470b7cf830.py::test_inputs_results2[values345] SKIPPED (Test ski...) [ 83%]
t_efce9a9f272c49c5a6a60c470b7cf830.py::test_inputs_results2[values000xfail] XFAIL (Test ...) [100%]

============================= 4 passed, 1 skipped